In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
csv_to_parquet('./train.csv', 'train')
csv_to_parquet('./test.csv', 'test')

# Data Load

In [3]:
train = pd.read_parquet('./train.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

# Data Pre-Processing

In [15]:
train_df = pd.DataFrame(train)
test_df = train_df[train_df['Delay'].isnull()]
test_df.drop(columns='Delay',inplace=True,axis=1)

In [19]:
airline_list = train_df[['Airline',"Carrier_Code(IATA)","Carrier_ID(DOT)"]]
airline_list.dropna(inplace=True)
airline_list.drop_duplicates(inplace=True)

In [20]:
# Carrier ID를 채울때 Airline 혹은 Carriercode가 같은 행의 데이터로 채운다.
def missing_values(df):
    missing_values = df["Carrier_ID(DOT)"].isnull()
    for idx, value in enumerate(missing_values):
        if value:
            subset = airline_list[(airline_list["Airline"] == df["Airline"][idx]) | (airline_list["Carrier_Code(IATA)"] == df["Carrier_Code(IATA)"][idx])]
            if len(subset) > 0:
                df.at[idx, "Carrier_ID(DOT)"] = subset["Carrier_ID(DOT)"].iloc[0]
                continue
    return df

train_df = missing_values(train_df)
test_df = missing_values(test_df)

KeyError: 302

In [ ]:
train_df['date'] = train_df['Month'].astype(str)+'-'+train_df['Day_of_Month'].astype(str)
train_df['date'] = pd.to_datetime(train_df['date'], format='%m-%d')
test_df['date'] = test_df['Month'].astype(str)+'-'+test_df['Day_of_Month'].astype(str)
test_df['date'] = pd.to_datetime(test_df['date'], format='%m-%d')